# Textanalyse

Die Verarbeitung natürlicher Sprache (Natural Language Processing oder NLP) ist ein Bereich der künstlichen Intelligenz (KI), der sich mit geschriebener und gesprochener Sprache befasst. Mit NLP können Sie Lösungen erstellen, die eine semantische Bedeutung aus Text oder Sprache extrahieren oder aussagekräftige Antworten in natürlicher Sprache formulieren.

Die *Cognitive Services* von Microsoft Azure umfassen den *Textanalysedienst*, der einige NLP-Funktionen vorkonfiguriert bereitstellt, beispielsweise zum Identifizieren von Schlüsselbegriffen in einem Text und zur Klassifizierung von Texten nach deren Stimmung.

![Ein Roboter liest ein Notizbuch](./images/NLP.jpg)

Angenommen, das fiktive Unternehmen *Margie's Travel* bittet seine Kunden, Bewertungen für Hotelaufenthalte zu schreiben. Der Textanalysedienst kann die Bewertungen zusammenfassen, indem er Schlüsselbegriffe extrahiert, positive und negative Bewertungen unterscheidet und den Bewertungstext nach Erwähnungen bekannter Entitäten wie etwa Orte und Personen durchsucht.

## Bewertungsdokumente anzeigen

Sehen Sie sich zunächst einige der Hotelbewertungen an, die von Kunden abgegeben wurden.

Die Bewertungen liegen als Textdateien vor. Um sie anzuzeigen, führen Sie den folgenden Code aus, indem Sie links neben der Zelle auf die Schaltfläche **Zelle ausführen** (&#9655;) klicken.

In [ ]:
import os

# Read the reviews in the /data/reviews folder
reviews_folder = os.path.join('data', 'text', 'reviews')

# Create a collection of reviews with id (file name) and text (contents) properties
reviews = []
for file_name in os.listdir(reviews_folder):
    review_text = open(os.path.join(reviews_folder, file_name)).read()
    review = {"id": file_name, "text": review_text}
    reviews.append(review)

for review_num in range(len(reviews)):
    # print the review text
    print('{}\n{}\n'.format(reviews[review_num]['id'], reviews[review_num]['text']))

## Erstellen einer Cognitive Services-Ressource

Sie können den Cognitive Service **Textanalyse** verwenden, um den Text in diesen Bewertungen zu analysieren. Dazu müssen Sie entweder eine **Textanalyse**- oder eine **Cognitive Services**-Ressource in Ihrem Azure-Abonnement bereitstellen. (Verwenden Sie eine Textanalyse-Ressource, falls dies der einzige Dienst ist, den Sie nutzen möchten oder falls Sie die Nutzung separat nachverfolgen möchten. Andernfalls können Sie eine Cognitive Services-Ressource verwenden, um den Textanalysedienst mit anderen Cognitive Services zu kombinieren und Entwicklern den Zugriff über eine einzige Kombination aus Endpunkt und Schlüssel zu ermöglichen.)

Falls noch nicht geschehen, führen Sie die folgenden Schritte aus, um eine **Cognitive Services**-Ressource in Ihrem Azure-Abonnement zu erstellen:

> **Hinweis**: Falls Sie bereits eine Cognitive Services-Ressource haben, können Sie die entsprechende **Schnellstart**-Seite im Azure-Portal öffnen und den Schlüssel und den Endpunkt der Ressource unten in die Zelle kopieren. Führen Sie andernfalls die folgenden Schritte aus, um eine Ressource zu erstellen.

1. Öffnen Sie das Azure-Portal unter „https://portal.azure.com“ in einer neuen Browserregisterkarte, und melden Sie sich mit Ihrem Microsoft-Konto an.
2. Klicken Sie auf die Schaltfläche **&#65291;Ressource erstellen**, suchen Sie nach *Cognitive Services*, und erstellen Sie eine **Cognitive Services**-Ressource mit den folgenden Einstellungen:
    - **Abonnement**: *Ihr Azure-Abonnement*
    - **Ressourcengruppe**: *Wählen Sie eine Ressourcengruppe aus, oder erstellen Sie eine Ressourcengruppe mit einem eindeutigen Namen.*
    - **Region**: *Wählen Sie eine verfügbare Region aus*:
    - **Name**: *Geben Sie einen eindeutigen Namen ein.*
    - **Tarif**: S0
    - **Ich bestätige, dass ich die Hinweise gelesen und verstanden habe**: Ausgewählt
3. Warten Sie, bis die Bereitstellung abgeschlossen ist. Öffnen Sie anschließend Ihre Cognitive Services-Ressource, und klicken Sie auf der Seite **Übersicht** auf den Link zur Schlüsselverwaltung für den Dienst. Sie benötigen den Endpunkt und Schlüssel, um sich aus Clientanwendungen heraus mit Ihrer Cognitive Services-Ressource zu verbinden.

### Abrufen des Schlüssels und Endpunkts für Ihre Cognitive Services-Ressource

Um Ihre Cognitive Services-Ressource verwenden zu können, benötigen Clientanwendungen deren Endpunkt und Authentifizierungsschlüssel:

1. Kopieren Sie im Azure-Portal auf der Seite **Schlüssel und Endpunkt** für Ihre Cognitive Service-Ressource den **Schlüssel1** für Ihre Ressource, und fügen Sie ihn im unten stehenden Code anstelle von **YOUR_COG_KEY** ein.
2. Kopieren Sie den **Endpunkt** für Ihre Ressource, und fügen Sie ihn unten im Code anstelle von **YOUR_COG_ENDPOINT** ein.
3. Führen Sie den Code in der folgenden Zelle aus, indem Sie auf die grüne Schaltfläche <span style="color:green">&#9655;</span> klicken.

In [ ]:
cog_key = 'YOUR_COG_KEY'
cog_endpoint = 'YOUR_COG_ENDPOINT'

print('Ready to use cognitive services at {} using key {}'.format(cog_endpoint, cog_key))

## Erkennen der Sprache
Zunächst werden wir die Sprache identifizieren, in der die Bewertungen verfasst wurden.

In [ ]:
import os
from azure.cognitiveservices.language.textanalytics import TextAnalyticsClient
from msrest.authentication import CognitiveServicesCredentials

# Get a client for your text analytics cognitive service resource
text_analytics_client = TextAnalyticsClient(endpoint=cog_endpoint,
                                            credentials=CognitiveServicesCredentials(cog_key))

# Analyze the reviews you read from the /data/reviews folder earlier
language_analysis = text_analytics_client.detect_language(documents=reviews)

# print detected language details for each review
for review_num in range(len(reviews)):
    # print the review id
    print(reviews[review_num]['id'])

    # Get the language details for this review
    lang = language_analysis.documents[review_num].detected_languages[0]
    print(' - Language: {}\n - Code: {}\n - Score: {}\n'.format(lang.name, lang.iso6391_name, lang.score))

    # Add the detected language code to the collection of reviews (so we can do further analysis)
    reviews[review_num]["language"] = lang.iso6391_name

## Extrahieren von Schlüsselbegriffen

Jetzt können Sie den Text in den Kundenbewertungen analysieren, um Schlüsselbegriffe zu identifizieren, die einen Hinweis auf die wichtigsten Themen liefern.

In [ ]:
# # Use the client and reviews you created in the previous code cell to get key phrases
key_phrase_analysis = text_analytics_client.key_phrases(documents=reviews)

# print key phrases for each review
for review_num in range(len(reviews)):
    # print the review id
    print(reviews[review_num]['id'])

    # Get the key phrases in this review
    print('\nKey Phrases:')
    key_phrases = key_phrase_analysis.documents[review_num].key_phrases
    # Print each key phrase
    for key_phrase in key_phrases:
        print('\t', key_phrase)
    print('\n')

Mit den Schlüsselbegriffen können Sie sich einen Überblick über die wichtigsten Themen in den einzelnen Bewertungen verschaffen. Bewertungen mit den Begriffen „hilfreiches Personal“ oder „schlechter Service“ liefern beispielsweise einen Hinweis darauf, was dem Verfasser besonders wichtig war.

## Ermitteln der Stimmung

Möglicherweise möchten Sie die Bewertungen anhand einer *Stimmungsbewertung* als *positiv* oder *negativ* klassifizieren. Dazu können Sie ebenfalls die Textanalyse verwenden.

In [ ]:
# Use the client and reviews you created previously to get sentiment scores
sentiment_analysis = text_analytics_client.sentiment(documents=reviews)

# Print the results for each review
for review_num in range(len(reviews)):

    # Get the sentiment score for this review
    sentiment_score = sentiment_analysis.documents[review_num].score

    # classifiy 'positive' if more than 0.5, 
    if sentiment_score < 0.5:
        sentiment = 'negative'
    else:
        sentiment = 'positive'

    # print file name and sentiment
    print('{} : {} ({})'.format(reviews[review_num]['id'], sentiment, sentiment_score))

## Extrahieren bekannter Entitäten

*Entitäten* sind Dinge, die im Text erwähnt werden und auf bekannte Elementarten verweisen. Dabei kann es sich beispielsweise um einen Ort, eine Person oder ein Datum handeln. Angenommen, Sie interessieren sich dafür, welche Daten und Orte in den Bewertungen erwähnt werden – dann können Sie dazu den folgenden Code verwenden.

In [ ]:
# Use the client and reviews you created previously to get named entities
entity_analysis = text_analytics_client.entities(documents=reviews)

# Print the results for each review
for review_num in range(len(reviews)):
    print(reviews[review_num]['id'])
    # Get the named entitites in this review
    entities = entity_analysis.documents[review_num].entities
    for entity in entities:
        # Only print datetime or location entitites
        if entity.type in ['DateTime','Location']:
            link = '(' + entity.wikipedia_url + ')' if entity.wikipedia_id is not None else ''
            print(' - {}: {} {}'.format(entity.type, entity.name, link))

Manche Entitäten sind so bekannt, dass sie eine eigene Wikipedia-Seite haben. In diesem Fall gibt der Textanalysedienst die URL für diese Seite zurück.

## Weitere Informationen

Weitere Informationen zur Textanalyse finden Sie in der [Dokumentation zur Textanalyse](https://docs.microsoft.com/azure/cognitive-services/text-analytics/).